In [9]:
class Door:

    def __init__(self, state="Closed"):
        self.state = state
        
    def set_state(self, state):
        self.state = state
        
    def get_state(self):
        return self.state

door = Door()
print(door.get_state())

Closed


In [7]:
class State:
    
    def __init__(self, state):
        self.state = state
        
    def enter(self):
        pass
    
    def exit(self):
        pass

In [21]:
class StateMachine:
    
    state_table = {
        ("open", "Opened"): ("Opened"),
        ("close", "Opened"): ("Closed"),
        ("open", "Closed"): ("Opened"),
        ("close", "Closed"): ("Closed"),
    }
    
    def __init__(self, obj):
        self.obj = obj
        
    def handle(self, event):
        current_state = self.obj.get_state()
        next_state = self.state_table.get((event, current_state))
        print(f"Handling event <{event}> in state <{current_state}>. Transit to state <{next_state}>")
        self.obj.set_state(next_state)
        

In [17]:
events = ['open', 'close', 'close', 'open', 'close', 'open']

door = Door()
machine = StateMachine(door)
for event in events:
    machine.handle(event)

Handling event <open> in state <Closed>. Transit to state <Opened>
Handling event <close> in state <Opened>. Transit to state <Closed>
Handling event <close> in state <Closed>. Transit to state <Closed>
Handling event <open> in state <Closed>. Transit to state <Opened>
Handling event <close> in state <Opened>. Transit to state <Closed>
Handling event <open> in state <Closed>. Transit to state <Opened>


In [18]:
class Predicate:
    
    def check(self, _input):
        if _input:
            return True
        return False

In [32]:
class StateMachineV2:
    
    predicate = Predicate()
    predicates = []
    predicates.append(predicate)
    state_table = {
        ("open", "Opened"): (predicates, ""),
        ("close", "Opened"): (predicates, "Closed"),
        ("open", "Closed"): (predicates, "Opened"),
        ("close", "Closed"): (predicates, ""),
    }
    
    def __init__(self, obj):
        self.obj = obj
        
    def handle(self, event, _input):
        current_state = self.obj.get_state()
        predicates, next_state = self.state_table.get((event, current_state))
        for predicate in predicates:
            if not predicate.check(_input):
                print(f"Handling event <{event}> in state <{current_state}>. Predicate fails")
                return
        
        if next_state:
            print(f"Handling event <{event}> in state <{current_state}>. Transit to state <{next_state}>")
            self.obj.set_state(next_state)
            return
        
        print(f"Handling event <{event}> in state <{current_state}>. Doing nothing")
        return

In [33]:
events = ['open', 'close', 'close', 'open', 'close', 'open']

door = Door()
machine = StateMachineV2(door)
for event in events:
    machine.handle(event, _input=False)
    
for event in events:
    machine.handle(event, _input=True)

Handling event <open> in state <Closed>. Predicate fails
Handling event <close> in state <Closed>. Predicate fails
Handling event <close> in state <Closed>. Predicate fails
Handling event <open> in state <Closed>. Predicate fails
Handling event <close> in state <Closed>. Predicate fails
Handling event <open> in state <Closed>. Predicate fails
Handling event <open> in state <Closed>. Transit to state <Opened>
Handling event <close> in state <Opened>. Transit to state <Closed>
Handling event <close> in state <Closed>. Doing nothing
Handling event <open> in state <Closed>. Transit to state <Opened>
Handling event <close> in state <Opened>. Transit to state <Closed>
Handling event <open> in state <Closed>. Transit to state <Opened>


So far, we have created following classes:
1. The `Door` class that represents `door` objects, doing nothing except providing method to change its own state.
2. The `StateMachine` which handles `event` to update `door`'s state based on `state_table`
3. The `state_table` defines a mapping between a combination of `event` and `current_state` to a `target_state`, along with a set of `predicates` that will validate the input before actually updating `door` state.
4. The `Predicate` class which defines a template for `predicates` with `check` method. The logic to validate the `input` will be encapsulated here.

A couple of thing to improve:
1. We only change the `door` state's label, there should be some activity going along with each particular state that we haven't implemented yet. We also haven't implemented a proper `State` class which handles these logic.
2. There's 2 approaches:
  - treats `Door` as `data class` which will have only methods to retrive/write data (with fundamental data validation such as min/max, empty/null values). Then, other business logic will be handled by the `StateMachine` and `State` class. `door` object will be passed around and manupulated accordingly.
  - Reusability:
    - `StateMachine` can be reused, given that we define a different `state_table` and move all business logic to `State` subclass.
    - `State` class is always reusable since it's only an interface. Business logic needs to reside in its subclass to cater different states of `door` object. If there is new object, such as a `VendorMachine`, we need to write new subclass.
  - Cons:
    - We can not encapsulate all activity of `Door` inside itself, make it more difficult to understand what a `Door` should do.
    - There's tight coupling between the `State` subclass and `door`'s internal. So, if the `door`'s internal changes, we need to update all `State` subclass accordingly.<br>
      A way to reduce this coupling is for the `Door` class implements and exposes all necessary methods to allow `State` works with `door` object instead of directly manipulating `door`'s internal attributes.

In [109]:
class Door:
    
    def __init__(self, material='Steal', has_alarms=True):
        assert isinstance(material, str), 'material must be a string'
        assert isinstance(has_alarms, bool), 'has_alarms must be a boolean'
        self.material = material
        self.has_alarms = has_alarms
        self.latch = "Disengaged"
        self.state = "Closed"
    
    def get_state(self):
        return self.state
    
    def set_state(self, state):
        self.state = state
        
        return None
    
    def alert(self):
        if self.has_alarms:
            print('Alert, alert!!!!')
            
        return None
    
    def lock(self):
        '''Lock the door'''
        if self.state == "Closed":
            print("Locking door")
            self.latch = "Engaged"
            self.state = "Locked"
        elif self.state == "Locked":
            print("Door is already locked")
        elif self.state == "Opened":
            print("The door is opened, cannot lock. Please close the door first.")
        else:
            raise Exception("Unknowed state")
        
    def unlock(self):
        '''Unlock the door'''
        if self.state == "Closed":
            print("Door is not locked")
        elif self.state == "Locked":
            print("Unlocking door")
            self.latch = "Disengaged"
            self.state = "Closed"
        elif self.state == "Opened":
            print("Door is opened, not locked")
        else:
            raise Exception("Unknowed state")
    
    def open(self):
        '''Open the door'''
        if self.state == "Closed":
            print("Openning door")
            self.state = "Opened"
        elif self.state == "Locked":
            print("Cannot open, door is locked. Unlock the door first.")
        elif self.state == "Opened":
            print("Door is already opened")
        else:
            raise Exception("Unknowed state")
        
    def close(self):
        '''Close the door'''
        if self.state == "Closed":
            print("Door is already closed")
        elif self.state == "Locked":
            print("Door is already closed and locked")
        elif self.state == "Opened":
            print("Closing door")
            self.state = "Closed"
        else:
            raise Exception("Unknowed state")
    
    def __repr__(self):
        return f'Door(material={self.material!r}, has_alamrs={self.has_alarms!r})'
    
    def __str__(self):
        return f"A {self.material} door that is currently {self.state}."

Okay, we re-write the `Door` class to give it some real life functionalities: `open()`, `close()`, `lock()`, and `unlock()`.

Possible states: `opened`, `closed`, `locked`

We also give it two new attributes:
- `material`: what is the door made from. This is for future use, such as whether it's flammable or how easy it is to break the door.
- `latch`: the door's latch that can be `engaged` or `disengaged`. We can only engage the latch when the door is `closed` and disengage when the door is `locked`.

The `__repr__()` and `__str__()` implementation is a good practice as they will provide sufficient information of a `door` object.

Let's generate some scenario to see how our door work.

In [79]:
import random

available_actions = ['close', 'open', 'lock', 'unlock']
actions = [random.choice(available_actions) for _ in range(10)]
    
door = Door('wooden', True)
print(door)
print("Scenario: ", actions)
for action in actions:
    getattr(door, action)()

A wooden door that is currently Closed.
Scenario:  ['close', 'close', 'lock', 'close', 'unlock', 'unlock', 'close', 'close', 'open', 'close']
Door is already closed
Door is already closed
Locking door
Door is already closed and locked
Unlocking door
Door is not locked
Door is already closed
Door is already closed
Openning door
Closing door


Look fine for now. Next, we will refactor this using State Pattern.

The general idea is that we will encapsulate the behaviors of a door when it's in a particular `state` to a separate class, i.e. `OpenedState`, `ClosedState`, and `LockedState`.
If we ever need to change how the door should behave when it's `Closed`, we only have to update the `ClosedState`, for example.

In [181]:
import abc

class State(abc.ABC):
    '''
    An abstract base class to provide all necessary state-specific behavior supported by Door class
    For each state-specific method in Door, we have an equivalent method in State
    Door.lock() <-> State.lock()
    It's always a good idea to use the same name for two relevant methods in the two classes
    to clearly express the intention of this pattern.
    
    We can either enforce subclass to implement all of these methods by make them @abstractmethod
    or provide a default implementation in case there's a lot of boiler plate code for subclass
    '''
    @property
    @abc.abstractmethod
    def state(self, door):
        pass
    
    def lock(self, door):
        '''Lock the door'''
        print(f"Cannot lock the door while door is {self.state}")
                
    def unlock(self, door):
        '''Unlock the door'''
        print(f"Cannot unlock the door while door is {self.state}")
        
    def open(self, door):
        '''Open the door'''
        print(f"Cannot open the door while door is {self.state}")
        
    def close(self, door):
        '''Close the door'''
        print(f"Cannot close the door while door is {self.state}")
        
    def __repr__(self):
        return f"{self.__class__.__qualname__}()"
    
    def __str__(self):
        return self.state

    
class OpenedState(State):
    
    @property
    def state(self):
        return "Opened"
    
    def close(self, door):
        print("Closing door.")
        door.set_state("Closed")
    

class ClosedState(State):
    
    @property
    def state(self):
        return "Closed"
    
    def open(self, door):
        print("Opening door.")
        door.set_state("Opened")
        
    def lock(self, door):
        print("locking door.")
        door.set_state("Locked")

class LockedState(State):
    
    @property
    def state(self):
        return "Locked"
    
    def open(self, door):
        print("Cannot open, door is locked. Unlock the door first.")
                
    def unlock(self, door):
        print("Unlocking door.")
        door.set_state("Closed")


In [163]:
class DoorV2:
    
    states = None
    
    def __init__(self, material='Steal', has_alarms=True):
        assert isinstance(material, str), 'material must be a string'
        assert isinstance(has_alarms, bool), 'has_alarms must be a boolean'
        self.material = material
        self.has_alarms = has_alarms
        self.latch = "Disengaged"
        self._state = None
        self.set_state()
    
    def get_state(self):
        if not self._state:
            self.set_state()
        return self._state
    
    def set_state(self, state="Closed"):
        if not self.states:
            self.states = {
                'Closed': ClosedState(),
                'Opened': OpenedState(),
                'Locked': LockedState(),
            }
        old_state = self._state
        self._state = self.states.get(state, ClosedState())
        
        print(f"Door was <{old_state}> now is <{self._state}>")
        
        return None
    
    def alert(self):
        if self.has_alarms:
            print('Alert, alert!!!!')
            
        return None
    
    def lock(self):
        '''Lock the door'''
        self._state.lock(self)
        
    def unlock(self):
        '''Unlock the door'''
        self._state.unlock(self)
    
    def open(self, _input):
        '''Open the door'''
        self._state.open(self)
        
    def close(self):
        '''Close the door'''
        self._state.close(self)
    
    def __repr__(self):
        return f'Door(material={self.material!r}, has_alamrs={self.has_alarms!r})'
    
    def __str__(self):
        return f"A {self.material} door that is currently {self._state}."

In [161]:
import random

available_actions = ['close', 'open', 'lock', 'unlock']
actions = [random.choice(available_actions) for _ in range(10)]
    
door = DoorV2('wooden', True)
print(door)
print("Scenario: ", actions)
for action in actions:
    print(f"Attempt to {action} while door is {door.get_state()}")
    getattr(door, action)()
    print('#########################################################')

Door was <None> now is <Closed>
A wooden door that is currently Closed.
Scenario:  ['unlock', 'lock', 'unlock', 'unlock', 'close', 'unlock', 'lock', 'open', 'close', 'unlock']
Attempt to unlock while door is Closed
Cannot unlock the door while door is Closed
#########################################################
Attempt to lock while door is Closed
locking door.
Door was <Closed> now is <Locked>
#########################################################
Attempt to unlock while door is Locked
Unlocking door.
Door was <Locked> now is <Closed>
#########################################################
Attempt to unlock while door is Closed
Cannot unlock the door while door is Closed
#########################################################
Attempt to close while door is Closed
Cannot close the door while door is Closed
#########################################################
Attempt to unlock while door is Closed
Cannot unlock the door while door is Closed
##############################

Look pretty neat, eh? `Door` basically does the following:
1. `set_state` and `get_state`, initialize `states` which is a dict that stores available states.
2. Delegate `open`, `close`, `lock`, `unlock` to appropriate `State` subclass.

## Analyze:
1. State-specific behaviors are encapsulated within its appropriate `State` subclass.
2. `State` base class provide a default implementation for all behavior which is a simple prompt and does nothing to the door. This could be replaced with an Exception if required. Subclass only needs to implement behaviors that are allowed in subclass's `state`, using the default implementation provided by base class `State` for other behaviors.
3. We have 4 new classes now. Changing behavior of a particular `state`, however, is simple.
4. Though, if there's new state, it will require more work since the states are slightly **coupling to each others** due to the fact that state transitions are also handled by state subclasses.<br>
`OpenedState` knows about `ClosedState` so upon `close()` the door, it could do:
```python
class OpenedState(State):
    ...
    def close(self, door):
        print("Closing door.")
        door.set_state("Closed")
    ...
```
If there's a new mediate state between `Opened` and `Closed`, say `PartiallyClosed`. We need to modify `OpenedState` class to update the transition (from `Opened -> Closed` to `Opened -> PartiallyClosed`, and we may also need to add new transition `Closed -> PartiallyClosed`.<br>
We will look at a different implementation using `table-driven` approach later.
5. Since the **transition logic is scattered** throughout subclasses, it's difficult to have a full picture of what's really going on, especially if we have more states and more behaviors.
6. Again, the transition between states in this example is quite simple, it still lacks some side-effect that a real-life scenario would have, such as making a sound as door is open, updating the record of how many times door was opened/closed, keep tabs or duration by which door is closed/opened/locked.

## Predicate and Input

One useful feature of a state machine has not being touched is the use of `Predicate` or a `Guarded Transition`. This allow the door to handle more complex scenarios based on various additional `inputs` we could provide, such as:
- whether the person is authorized to open the door. The `input` in this case could be the person's id provided by a card reader, finger print scanner, eye scanner or face recognition.
- A surveillance camera that could tell whether the person is tailgated that result in door won't be opened.
- With thermometers, such camera could also tell whether the person is sick via body temperature to determine if this person is allowed to enter
We should be able to implement all of this feature without the `State Pattern` but you could imagine how complex and maybe confused your code could become with all of these logic.

Below is a simple implementation of `Predicate`

In [4]:
import abc


class DoorInput:
    
    def __init__(self, being_followed=False, being_sick=False, is_authorized=True):
        self.being_followed = being_followed
        self.being_sick = being_sick
        self.is_authorized = is_authorized
        
class Predicate(abc.ABC):
    
    @abc.abstractmethod
    def check(self, _input):
        return False
    
class TailgatePredicate(Predicate):
    
    def check(self, _input: DoorInput):
        assert isinstance(_input, DoorInput)
        if not _input.being_followed:
            print("Not being followed, please proceed")
            return True
        
        print("Being followed, cannot proceed")
        return False

In [5]:
import abc

class StateV2(abc.ABC):
    '''
    An abstract base class to provide all necessary state-specific behavior supported by Door class
    For each state-specific method in Door, we have an equivalent method in State
    Door.lock() <-> State.lock()
    It's always a good idea to use the same name for two relevant methods in the two classes
    to clearly express the intention of this pattern.
    
    We can either enforce subclass to implement all of these methods by make them @abstractmethod
    or provide a default implementation in case there's a lot of boiler plate code for subclass
    '''
    @property
    @abc.abstractmethod
    def state(self, door):
        pass
    
    def lock(self, door, *args, **kwargs):
        '''Lock the door'''
        print(f"Cannot lock the door while door is {self.state}")
                
    def unlock(self, door, *args, **kwargs):
        '''Unlock the door'''
        print(f"Cannot unlock the door while door is {self.state}")
        
    def open(self, door, *args, **kwargs):
        '''Open the door'''
        print(f"Cannot open the door while door is {self.state}")
        
    def close(self, door, *args, **kwargs):
        '''Close the door'''
        print(f"Cannot close the door while door is {self.state}")
        
    def __repr__(self):
        return f"{self.__class__.__qualname__}()"
    
    def __str__(self):
        return self.state
    
    
class ClosedState(StateV2):
    
    predicates = [
        TailgatePredicate()
    ]
    
    @property
    def state(self):
        return "Closed"
    
    def open(self, door, _input):
        for predicate in self.predicates:
            if not predicate.check(_input):
                return
        print("Opening door.")
        door.set_state("Opened")
        
    def lock(self, door, *args, **kwargs):
        print("locking door.")
        door.set_state("Locked")
        
class OpenedState(StateV2):
    
    @property
    def state(self):
        return "Opened"
    
    def close(self, door, *args, **kwargs):
        print("Closing door.")
        door.set_state("Closed")


class LockedState(StateV2):
    
    @property
    def state(self):
        return "Locked"
    
    def open(self, door, *args, **kwargs):
        print("Cannot open, door is locked. Unlock the door first.")
                
    def unlock(self, door, *args, **kwargs):
        print("Unlocking door.")
        door.set_state("Closed")

In [18]:
class DoorV3:
    
    states = None
    
    def __init__(self, material='Steal', has_alarms=True):
        assert isinstance(material, str), 'material must be a string'
        assert isinstance(has_alarms, bool), 'has_alarms must be a boolean'
        self.material = material
        self.has_alarms = has_alarms
        self.latch = "Disengaged"
        self._state = None
        self.set_state()
    
    def get_state(self):
        if not self._state:
            self.set_state()
        return self._state
    
    def set_state(self, state="Closed"):
        if not self.states:
            self.states = {
                'Closed': ClosedState(),
                'Opened': OpenedState(),
                'Locked': LockedState(),
            }
        old_state = self._state
        self._state = self.states.get(state, ClosedState())
        
        print(f"Door was <{old_state}> now is <{self._state}>")
        
        return None
    
    def alert(self):
        if self.has_alarms:
            print('Alert, alert!!!!')
            
        return None
    
    def lock(self, _input):
        '''Lock the door'''
        self._state.lock(self)
        
    def unlock(self, _input):
        '''Unlock the door'''
        self._state.unlock(self)
    
    def open(self, _input):
        '''Open the door'''
        self._state.open(self, _input)
        
    def close(self, _input):
        '''Close the door'''
        self._state.close(self)
    
    def __repr__(self):
        return f'Door(material={self.material!r}, has_alamrs={self.has_alarms!r})'
    
    def __str__(self):
        return f"A {self.material} door that is currently {self._state}."

In [19]:
import random

available_actions = ['close', 'open', 'lock', 'unlock']
actions = [random.choice(available_actions) for _ in range(10)]
    
door = DoorV3('wooden', True)
print(door)
print("Scenario: ", actions)
_input = DoorInput(being_followed=True)

actions = ['open', 'close']
for action in actions:
    print(f"Attempt to {action} while door is {door.get_state()}")
    getattr(door, action)(_input)
    print('#########################################################')

Door was <None> now is <Closed>
A wooden door that is currently Closed.
Scenario:  ['unlock', 'lock', 'lock', 'close', 'unlock', 'unlock', 'lock', 'lock', 'open', 'lock']
Attempt to open while door is Closed
Being followed, cannot proceed
#########################################################
Attempt to close while door is Closed
Cannot close the door while door is Closed
#########################################################


# Table-driven state machine

Core idea of this approach is to create a table that will determine the `target_state`, `transition`, and `predicates` based on `event`, `input`, and `current_state`. The table will look something like this:

```python
state_table = {
    (event, InputClass, current_state): (predicates, transition, target_state),
    (event2, InputClass, current_state): (predicates2, transition2, target_state),
    ...
}
```
In this particular implementation, `state_table` is a `dict`. `event`, `InputClass`, and `current_state` together form a `tuple` that is a `key` in `state_table`.<br>

*Note*: You might want to use `InputClass` or even `EventClass` instead of a pre-instantiated object to define transitions to make it usable for different `input` of the same type (same class). You could also use a base class to have more variety.

In other words, with any given combination of `event`, `input`, `current_state`, we can form a `tuple` with which we look up `state_table` to retrieve corresponding `target_state`, `transition`, and `predicates` then finally execute them in following order:
1. Check/validate `input` by running `predicates`. Some of the business logic, mostly validation will be implemented in `predicate`
2. If all the checks pass, execute `transition` which implements the main chunk of business logic and other side effects, if any.
3. Update the object to `target_state`, so the next event coming in will be processed based on this new `target_state`


## Analysis
- The `state` only serves as a token. It doesn't actually do anything. It's only use is to look up the appropriate transitions.
- The `input` play two important roles:
  1. as a token, just like `state` to form the key of `state_table`.
  2. Its content could be used further by `predicates` or `transition`.
    - it could be some `input` that we need validate to proceed
    - `transition` will need `input` that give information as to how we can update the machine's internal state:
    
    | Input                                     | Used for                   |
    | :-------------                            | :----------:               |
    | amount of money input to vending machine  | tally up the total amount  |
    | floor number input to elevator            | determine next destination |
    
- `predicates` could be encapsulated in different classes and to be used as composition.
- `transition` could be an internal method of the machine or an external class/method. Consider bundling `transition` along with `state` if they're closely related to each other.

With this setup, all possible transitions between one state to another is centralized in one place, making it easy to understand the flow. It's also very flexible in definine transitions, since you could have different combinations to handle different scenarios:
### Multiple events for one state
```python
state_table = {
    (event, InputClass, current_state): (predicates, transition, target_state),
    (event2, InputClass, current_state): (predicates2, transition2, target_state2),
    (event3, InputClass, current_state): (predicates3, transition3, target_state3),
    ...
}
```
We could mix with different `input` as well:
```python
state_table = {
    (event, InputClass, current_state): (predicates, transition, target_state),
    (event2, InputClass2, current_state): (predicates2, transition2, target_state2),
    (event3, InputClass, current_state): (predicates3, transition3, target_state3),
    ...
}
```
### Multiple target_state for a set of event and input
```python
state_table = {
    (event, InputClass, current_state): [
        (predicates, transition, target_state),
        (predicates2, transition2, target_state2),
        (predicates3, transition3, target_state3),
    ]
    ...
}
```
In this scenario, depends on the result of `predicates` run, we will transit to appropriate `target_state`. Note that it will check in a certain order (e.g. `predicates` -> `predicates2` -> `predicate3` and so on if there's more) and trigger the corresponding transition then exit, so first come first serve. Make sure that there's no overlapping `predicates`, i.e. one set of `input` will only satisfy one `predicate`. Otherwise, you have to ensure the order of `predicates` is appropriate for your need. This will make the table become much more complex and difficult to predict the outcome though.

### Do some side effect with current_state but not changing state
```python
state_table = {
    (event, InputClass, current_state): (predicates, transition, None),
    ...
}
```
This setup will check `predicates` then run `transition` to have some desired side effect still keep the `current_state` afterward (`target_state` equals `None`).

Giving a Vending Machine as an example, the Machine will have a state in which it collects money, naming `collect`. While in `collect` state, some possible scenarios are:
- We give more money -> the machine should tally up total money so far, and remaining in `collect` state
- We select items -> the machine will now switch to `select` state

In case of an elevator, when we press the open door button, the elavator door should be opened, and the state now is `waiting_for_people_to_come`, if we press the open door button again, it's still doing the samething and remains in same `waiting` state.

**Cons**:
- State table will grow exponentially as you have more states, events, and inputs since the table defines the combinations of these. Having a default will help to mitigate this, e.g. to raise an exception for those combinations that are not defined. You only need to define the necessary transitions.
- There's more than one approach for a concrete implementation, so you need to be clear and thorough with your approach. Avoiding mixing different approach will make your code more comprehensible and clean.
- We might need to define a general common interface for `input` and `predicates`.

In [11]:
class DoorV4:
    
    
    # This state table defines all combination of event + input + current state
    # for each combination, we will have a corresponding set of predicates + transition + target state
    # The handle_event() will take in an event along with some inputs, combine with the current state,
    # we will be able to find appropriate predicates, transition and target state
    
        
    def __init__(self, material='Steal', has_alarms=True):
        assert isinstance(material, str), 'material must be a string'
        assert isinstance(has_alarms, bool), 'has_alarms must be a boolean'
        self.material = material
        self.has_alarms = has_alarms
        self.latch = "Disengaged"
        self._state = ClosedState
    
    def get_state(self):
        return self._state
    
    def set_state(self, state=ClosedState):
        old_state = self._state
        self._state = state
        
        print(f"Door was <{old_state}> now is <{self._state}>")
        
        return None
    
    def handle_event(self, event, _input):
        predicates, transition, target_state = self.state_table.get((event, _input.__class__, self.get_state()), (None, None, None))
        if transition:
            for predicate in predicates:
                if not predicate.check(_input):
                    return

            transition(self)
        
        if target_state:
            self.set_state(target_state)

        return
        
    def alert(self):
        if self.has_alarms:
            print('Alert, alert!!!!')
            
        return None
    
    def lock(self):
        '''Lock the door'''
        self._state.lock(self)
        
    def unlock(self):
        '''Unlock the door'''
        self._state.unlock(self)
    
    def open(self):
        '''Open the door'''
        print('Opening door')
        
    def close(self):
        '''Close the door'''
        self._state.close(self)
    
    def __repr__(self):
        return f'Door(material={self.material!r}, has_alamrs={self.has_alarms!r})'
    
    def __str__(self):
        return f"A {self.material} door that is currently {self._state}."
    
DoorV4.state_table = {
        ('open', DoorInput, ClosedState): ([TailgatePredicate()], DoorV4.open, OpenedState),
        ('close', DoorInput, ClosedState): ([TailgatePredicate()], None, None),
    }
    

In [14]:
import random

available_actions = ['close', 'open', 'lock', 'unlock']
actions = [random.choice(available_actions) for _ in range(10)]
    
door = DoorV4('wooden', True)
print(door)
print("Scenario: ", actions)


for action in actions:
    _input = DoorInput(being_followed=random.choice([0, 1]))
    print(f"Attempt to {action} while door is {door.get_state()}")
    door.handle_event(action, _input)
    print('#########################################################')

A wooden door that is currently <class '__main__.ClosedState'>.
Scenario:  ['unlock', 'open', 'close', 'close', 'open', 'open', 'close', 'unlock', 'lock', 'unlock']
Attempt to unlock while door is <class '__main__.ClosedState'>
#########################################################
Attempt to open while door is <class '__main__.ClosedState'>
Not being followed, please proceed
Opening door
Door was <<class '__main__.ClosedState'>> now is <<class '__main__.OpenedState'>>
#########################################################
Attempt to close while door is <class '__main__.OpenedState'>
#########################################################
Attempt to close while door is <class '__main__.OpenedState'>
#########################################################
Attempt to open while door is <class '__main__.OpenedState'>
#########################################################
Attempt to open while door is <class '__main__.OpenedState'>
#############################################